In [1]:
import pandas
data = pandas.read_csv('data/train.csv')

In [4]:
from sklearn.preprocessing import OneHotEncoder
import numpy
from sklearn.impute import SimpleImputer

data = pandas.read_csv('data/train.csv')
X = data.select_dtypes(exclude=['object']).drop('SalePrice', axis=1)
y = data.SalePrice
my_imputer = SimpleImputer(strategy='mean')
X_imputed = my_imputer.fit_transform(X)
X_categorical = data.select_dtypes(include=['object'])
my_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_imputed = my_imputer.fit_transform(X_categorical)
my_imputer = SimpleImputer(strategy='most_frequent')
X_categorical_imputed = my_imputer.fit_transform(X_categorical)
encoder = OneHotEncoder(handle_unknown='ignore')
encoded = encoder.fit(X_categorical_imputed)
transformed = encoded.fit_transform(X_categorical_imputed)
merged = numpy.concatenate((X_imputed, transformed.toarray()), axis=1)
merged_columns = [c for c in X.columns] + [c for c in encoder.get_feature_names()]
merged_data_frame = pandas.DataFrame(data=merged, columns=merged_columns)

In [6]:
merged_data_frame.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,x41_ConLw,x41_New,x41_Oth,x41_WD,x42_Abnorml,x42_AdjLand,x42_Alloca,x42_Family,x42_Normal,x42_Partial
0,1.0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,2.0,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3.0,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.0,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5.0,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
